In [1]:
import tensorflow as tf 
from keras import backend as K
from keras.optimizers import SGD
from keras.models import load_model, Model
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import pydicom
import cv2
from scipy.ndimage import zoom
from sklearn import preprocessing

In [2]:
train_data = pd.read_csv(f"D:/Downloads/rsna-2023-abdominal-trauma-detection/train.csv")
train_data.head()

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1


In [3]:
def get_data_for_3d_volumes(data, path, number_idx):
    
    data_to_merge = data[["patient_id", "any_injury"]]
    shuffled_data = data_to_merge.sample(frac=1, random_state=42)
    shuffled_indexes = shuffled_data.index[:number_idx]
    selected_rows = shuffled_data.loc[shuffled_indexes]
    data_to_merge_processed = selected_rows.reset_index()
    
    total_paths = []
    patient_ids = []
    category = []
    
    for patient_id in range(len(data_to_merge_processed)):
    
        p_id = str(data_to_merge_processed["patient_id"][patient_id])
        str_imgs_path = path + p_id + '/'

        patient_img_paths = []

        for file in glob(str_imgs_path + '*'):
            for image_path in glob(file + '/*'):
                 patient_img_paths.append(image_path)
    
        total_paths.append(patient_img_paths)
        patient_ids.append(data_to_merge_processed["patient_id"][patient_id])
        category.append(data_to_merge_processed["any_injury"][patient_id])
    
    final_data = pd.DataFrame(list(zip(patient_ids, total_paths, category)),
               columns =["Patient_id", "Patient_paths", "Patient_category"])
    
    return final_data  

In [4]:
path = 'D:/Downloads/rsna-2023-abdominal-trauma-detection/train_images/'

paths = get_data_for_3d_volumes(train_data, path=path, number_idx=5)

In [5]:
paths.head()

,Patient_id,Patient_paths,Patient_category
0,65310,[D:/Downloads/rsna-2023-abdominal-trauma-detec...,0
1,27897,[D:/Downloads/rsna-2023-abdominal-trauma-detec...,0
2,35348,[D:/Downloads/rsna-2023-abdominal-trauma-detec...,0
3,32425,[D:/Downloads/rsna-2023-abdominal-trauma-detec...,1
4,6968,[D:/Downloads/rsna-2023-abdominal-trauma-detec...,0


## Definition of the 3D CNN model

In [6]:
# 3D CNN MODEL
def convolutional_block_3d(inputs, num_filters):

    x = Conv3D(filters=num_filters, kernel_size=(3,3,3),
    activation="relu")(inputs)
    x = MaxPool3D(pool_size=(2,2,2))(x)
    x = BatchNormalization()(x)

    return x

# function to define the dense block of the network, composed by:
# 2 dense layer with 2 dropout layes in between and one output layer for clasification
def dense_block(flatten_layer):
    dense_layer_1 = Dense(units=512, activation='relu')(flatten_layer)
    dense_layer_1 = Dropout(0.4)(dense_layer_1)

    #dense_layer_2 = Dense(units=256, activation='relu')(dense_layer_1)
    #dense_layer_2 = Dropout(0.4)(dense_layer_2)
    output_layer = Dense(units=2, activation='softmax')(dense_layer_1)

    return output_layer

# Main function to build the 3D Conv Network
def build_3d_network(input_shape):

    input_layer = Input(input_shape)

    x1 = convolutional_block_3d(input_layer, 64)
    x2 = convolutional_block_3d(x1, 64)
    x3 = convolutional_block_3d(x2, 128)
    x4 = convolutional_block_3d(x3, 256)

    flatten_layer = Flatten()(x4)

    output = dense_block(flatten_layer)
    model = Model(inputs=input_layer, outputs=output)

    model.compile(loss='mae',optimizer=SGD(learning_rate=1e-06, momentum=0.99, decay=0.0, nesterov=False), metrics=['acc'])

    return model

## Data preprocessing -> Generating 3D volumes from patient CT scans

In [7]:
# Resizing images 
def resize_img(img_paths, size=(128, 128)):
    preprocessed_images = []
    for image_path in img_paths: 
        image = pydicom.read_file(image_path)
        image = image.pixel_array
        #if image.shape == (512, 512):
        image = cv2.resize(image, size)
        image_array = np.array(image)
        preprocessed_images.append(image_array)

# Create an empty volume array
    volume_shape = (size[0], size[1], len(preprocessed_images)) 
    volume = np.zeros(volume_shape, dtype=np.uint16)
# Populate the volume with images
    for i, image_array in enumerate(preprocessed_images):
        volume[:,:,i] = image_array
        return volume

# Implementation of SIZ algorithm
def change_depth_siz(patient_volume):
    desired_depth = 64
    current_depth = patient_volume.shape[-1]
    depth = current_depth / desired_depth
    depth_factor = 1 / depth
    img_new = zoom(patient_volume, (1, 1, depth_factor), mode='nearest')
    return img_new

## Generating 3D Volumes

In [8]:
def generate_volume_for_patients_and_labels(patient_paths, patient_labels):

    volumes = []
    labels = []
    
    for i, list_path in enumerate(patient_paths):
        patient_volume = resize_img(list_path)
        patient_volume = change_depth_siz(patient_volume)
        
        original_shape = patient_volume.shape
        flattened_image = patient_volume.reshape((-1,))
        
        scaler = preprocessing.MinMaxScaler()
        normalized_flattened_image = scaler.fit_transform(flattened_image.reshape((-1, 1)))
        
        normalized_volume_image = normalized_flattened_image.reshape(original_shape)
        volumes.append(normalized_volume_image)
        
    for n, patient_label in enumerate(patient_labels):
        features = patient_label
        labels.append(features)
        
    return np.array(volumes), np.array(labels)

In [9]:
volumes_from_patients, labels = generate_volume_for_patients_and_labels(paths["Patient_paths"], paths["Patient_category"])

In [10]:
volumes_from_patients.shape

(5, 128, 128, 64)

In [11]:
labels.shape

(5,)

## Model Training

In [12]:
input_shape = (128, 128, 64, 1)

model = build_3d_network(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 64, 1)  0         
                             ]                                   
                                                                 
 conv3d (Conv3D)             (None, 126, 126, 62, 64)  1792      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 63, 63, 31, 64)   0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 63, 63, 31, 64)   256       
 ormalization)                                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 61, 61, 29, 64)    110656    
                                                             

In [13]:
model.fit(volumes_from_patients, labels, epochs=10, batch_size=4)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/max_pooling3d/MaxPool3D/MaxPool3DGrad' defined at (most recent call last):
    File "c:\Users\Daniel\miniconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Daniel\miniconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Daniel\miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Daniel\miniconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\Daniel\miniconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "c:\Users\Daniel\miniconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\User